In [ ]:
!pip3 install "transformers[torch]" sentencepiece datasets pandas accelerate evaluate

In [2]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset

## Cargamos nuestro dataset
Usaremos cafeteria.json que tiene datos sobre el menú de todas las cafeterías de la UAM

In [ ]:
# Cargar archivo cafeteria.json
with open("cafeteria.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Verificar primeras entradas
data[:3]

## Creamos la clase de CafeteriaDataset de PyTorch 

In [4]:
class CafeteriaDataset(Dataset):
    def __init__(self, data, tokenizer, max_input=64, max_output=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input = max_input
        self.max_output = max_output

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]["question"]
        answer = self.data[idx]["answer"]

        # Tokenizar input (pregunta)
        input_enc = self.tokenizer(
            "Pregunta: " + question,
            truncation=True,
            padding="max_length",
            max_length=self.max_input,
            return_tensors="pt",
        )

        # Tokenizar target (respuesta)
        target_enc = self.tokenizer(
            answer,
            truncation=True,
            padding="max_length",
            max_length=self.max_output,
            return_tensors="pt",
        )

        return {
            "input_ids": input_enc["input_ids"].squeeze(),
            "attention_mask": input_enc["attention_mask"].squeeze(),
            "labels": target_enc["input_ids"].squeeze(),
        }

## Preparamos el modelo y tokenizamos

In [ ]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Detectar GPU Apple (MPS) o usar CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Usando dispositivo: {device}")
model.to(device)

## Preparamos el dataset

In [6]:
dataset = CafeteriaDataset(data, tokenizer)

## Configuramos el entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # pequeño para demo
    num_train_epochs=10,
    logging_steps=5,
    save_total_limit=1,
    remove_unused_columns=False,  # importante para T5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

## Ahora entrenamos

In [ ]:
trainer.train()

## Guardamos el modelo

In [ ]:
model.save_pretrained("./modelo_cafeteria_t5")
tokenizer.save_pretrained("./modelo_cafeteria_t5")
print("Modelo entrenado y guardado en ./modelo_cafeteria_t5")

## Nuestro modelo ya estaría listo y entrenado ahora solo haría falta probarlo de la siguiente forma:

- Cargamos el modelo que se ha creado
- Lo probamos con la siguiente función

In [10]:
def cargar_modelo_y_responder(model_path="./modelo_cafeteria_t5"):
    """
    Carga el modelo T5 entrenado y devuelve una función para responder preguntas.
    """
    # Cargar modelo y tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    model.to(device)
    model.eval()

    def responder(pregunta):
        input_enc = tokenizer(
            "Pregunta: " + pregunta,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=64,
        )
        input_enc = {k: v.to(device) for k, v in input_enc.items()}
        outputs = model.generate(
            **input_enc,
            max_new_tokens=64,
            num_beams=2,
            early_stopping=True,
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return responder

Pruebas:

In [ ]:
responder = cargar_modelo_y_responder()

print(responder("¿Cuánto cuesta el café con leche?"))
print(responder("¿Qué horario tiene la cafetería?"))
print(responder("¿Hay opciones sin gluten?"))